# RAG - ColBERT IR, gpt tubo 3.5

## IR

### 라이브러리 설치 및 임포트

In [2]:
# 필요한 라이브러리 import

from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection
import colbert
import json

In [3]:
with open("/data/ephemeral/home/upstage-ai-advanced-ir2/data/documents.jsonl") as f:
    docs = [json.loads(line) for line in f]

In [4]:
# docs 에서 줄바꿈 문자를 제거 (학습 할때 에러 방지)
for doc in docs:
    doc['content'] = doc['content'].replace("\n", "")
    doc['content'] = doc['content'].replace("\r", "")

In [ ]:
print(docs[0])

In [ ]:
collection = [doc['content'] for doc in docs]
collection[0]

In [ ]:
docid_list = [doc['docid'] for doc in docs]
docid_list[0]

In [ ]:
src_list = [doc['src'] for doc in docs]
src_list[0]

### ColBER 색인

In [ ]:
import torch
print(torch.cuda.is_available())  # True여야 합니다.
print(torch.cuda.get_device_name(0))  # 'Tesla T4'와 같은 GPU 이름이 나와야 합니다.


In [ ]:
import torch
print(torch.cuda.is_available())  # True가 출력되면 GPU 사용이 가능함을 의미합니다.
print(torch.cuda.device_count())  # 사용 가능한 GPU의 수를 출력합니다.


In [11]:
# 위에서 확인한 학습된 모델의 위치를 checkpoint에 넣어줌
checkpoint = '/data/ephemeral/home/upstage-ai-advanced-ir2/experiments/sentence-transformer-klue/none/2024-10/14/13.45.30/checkpoints/colbert'
experiment = 'after_trained'
index_name = 'science_common_sense'
nbits = 2

In [ ]:


with Run().context(RunConfig(nranks=1, experiment=experiment)):  # nranks specifies the number of GPUs to use
    config = ColBERTConfig(nbits=nbits, kmeans_niters=4) # kmeans_niters specifies the number of iterations of k-means clustering; 4 is a good and fast default.
                                                                                # Consider larger numbers for small datasets.
    indexer = Indexer(checkpoint = checkpoint, config = config)
    indexer.index(name = index_name, collection = collection, overwrite=True)

In [ ]:
# 검색을 수행하기 위한 Searcher 객체를 생성합니다.
# ColBERT 인덱스를 기반으로 검색 작업을 수행합니다.

with Run().context(RunConfig(experiment=experiment)):  # 실행 환경을 설정합니다. 여기서는 'notebook'이라는 이름으로 실험을 정의합니다.
    searcher = Searcher(index=index_name, collection=collection)  # 지정된 인덱스와 컬렉션을 사용하여 Searcher 객체를 초기화합니다.


### Colbert 색인 문서 검색

In [ ]:
# 문서 검색 test
queries = ["나무 분류하는 방법은?"]  # 검색할 질의 목록입니다.

# 각 질의에 대해 검색을 수행합니다.
for query in queries:
    print(f"#> {query}")  # 현재 검색 질의를 출력합니다.

    # 검색 질의에 대한 상위 5개의 문서를 검색합니다.
    results = searcher.search(query, k=3)  # k는 검색 결과에서 반환할 상위 문서의 개수를 의미합니다.

    # 검색된 상위 k개의 문서를 출력합니다.
    for passage_id, passage_rank, passage_score in zip(*results):
        # 검색 결과의 순위, 점수 및 해당 문서를 출력합니다.
        print(f"{docid_list[passage_id]}\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")


In [ ]:
import json

# 파일 경로 설정
eval_file = '/data/ephemeral/home/upstage-ai-advanced-ir2/data/eval.jsonl'
output_file = '/data/ephemeral/home/upstage-ai-advanced-ir2/submission_test/output.jsonl'

# 새로 저장할 데이터를 담을 리스트
output_data = []

# eval.jsonl 파일 읽기
with open(eval_file, 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line.strip())
        
        # eval_id와 content (standalone_query)를 추출
        eval_id = data.get('eval_id')
        if 'msg' in data and len(data['msg']) > 0:
            standalone_query = data['msg'][0]['content']  # 첫 번째 메시지의 'content' 값을 standalone_query로 사용

            # 검색 작업 수행 (예시로 상위 3개 문서 검색)
            print(f"#> {standalone_query}")  # 검색 질의 출력
            results = searcher.search(standalone_query, k=3)  # 상위 3개의 문서 검색
            
            # 검색된 상위 k개의 문서를 출력
            topk = []
            for passage_id, passage_rank, passage_score in zip(*results):
                docid = docid_list[passage_id]
                topk.append(docid)  # 상위 문서 ID 저장
                # 검색 결과 출력
                print(f"{docid}\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")
            
            # 새 데이터 형식으로 변환하여 리스트에 추가
            output_data.append({
                'eval_id': eval_id,
                'standalone_query': standalone_query,
                'topk': topk
            })

# 결과를 jsonl 형식으로 파일에 저장
with open(output_file, 'w', encoding='utf-8') as f_out:
    for entry in output_data:
        f_out.write(json.dumps(entry, ensure_ascii=False) + '\n')

print(f"Data successfully saved to {output_file}")


## RAG

### 필요 라이브러리 임포트 및 openai api 준비하기

In [ ]:
%pip install openai

In [17]:
# 아래부터는 실제 RAG를 구현하는 코드입니다.
from openai import OpenAI  # OpenAI API를 사용하기 위한 모듈 가져오기
import traceback  # 예외 발생 시 오류 추적을 위한 모듈 가져오기
from dotenv import load_dotenv
import os

# OPENAI_API_KEY.env 파일 생성

# OPENAI_API_KEY = "your API key here"

# 특정 .env 파일 경로를 지정하여 환경 변수 로드
load_dotenv(dotenv_path='/data/ephemeral/home/upstage-ai-advanced-ir2/OPENAI_API_KEY.env')

# OpenAI API 키 가져오기
openai_api_key = os.getenv("OPENAI_API_KEY")

# 이후 OpenAI API 키를 사용할 코드 작성
# print(openai_api_key)  # API 키를 확인하기 위한 출력 (실제 코드에서는 제거)
client = OpenAI()  # OpenAI 클라이언트 생성
# 사용할 모델을 설정(여기서는 gpt-3.5-turbo-1106 모델 사용)
llm_model = "gpt-3.5-turbo-1106"

In [18]:
# RAG 구현에 필요한 Question Answering을 위한 LLM 프롬프트
persona_qa = """
## Role: 과학 상식 전문가

## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 한국어로 답변을 생성한다.
"""

# RAG 구현에 필요한 질의 분석 및 검색 이외의 일반 질의 대응을 위한 LLM 프롬프트
persona_function_calling = """
## Role: 과학 상식 전문가

## Instruction
- 사용자가 대화를 통해 과학 지식에 관한 주제로 질문하면 search api를 호출할 수 있어야 한다.
- 과학 상식과 관련되지 않은 나머지 대화 메시지에는 적절한 대답을 생성한다.
"""

# Function calling에 사용할 함수 정의
tools = [
    {
        "type": "function",  # 함수 타입 설정
        "function": {
            "name": "search",  # 함수 이름 설정
            "description": "search relevant documents",  # 함수 설명 설정
            "parameters": {
                "properties": {  # 파라미터의 속성 설정
                    "standalone_query": {
                        "type": "string",  # 파라미터의 데이터 타입을 'string'으로 설정
                        # "description": "Final query suitable for use in search from the user messages history."  # 사용자 메시지 히스토리로부터 검색에 사용할 최종 질의 설명
                        "description": "사용자 메세지 기록으로부터 검색에 사용할 적합한 최종 질의 설명"
                    }
                },
                "required": ["standalone_query"],  # 필수 파라미터로 'standalone_query' 설정
                "type": "object"  # 파라미터 전체 타입을 'object'로 설정
            }
        }
    },
]


In [19]:
# 대화형 에이전트를 위한 대화 관리자(dialog manager) 구현

# 대화형 에이전트가 사용자와 상호작용하면서 검색 기능을 수행하는 함수입니다.
def conversation_search(messages, persona, tools):
    # 시스템 메시지와 사용자 메시지를 결합하여 메시지 리스트를 생성합니다.
    msg = [{"role": "system", "content": persona}] + messages

    # OpenAI의 Chat API를 사용하여 대화 응답을 생성합니다.
    result = client.chat.completions.create(
        model=llm_model,  # 사용할 모델을 지정합니다.
        messages=msg,  # 구성된 메시지 리스트를 전달합니다.
        tools=tools,  # 사용할 도구 목록을 전달합니다.
        temperature=0,  # 답변의 일관성을 높이기 위해 온도를 0으로 설정합니다.
        seed=1  # 결과의 재현성을 유지하기 위해 랜덤 시드를 설정합니다.
    )

    # 생성된 응답 메시지를 변수에 저장합니다.
    response_message = result.choices[0].message

    # 만약 응답 메시지에서 도구 호출이 포함된 경우
    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            # 도구 호출에 전달된 인자를 JSON 형식으로 파싱합니다.
            function_args = json.loads(tool_call.function.arguments)
            standalone_query = function_args.get("standalone_query")  # 독립적인 검색 쿼리를 추출합니다.

            # 검색 쿼리를 이용해 상위 3개의 결과를 검색합니다.
            results = searcher.search(standalone_query, k=3)
            print("검색어", standalone_query)
            print()
            print("검색 결과 :")
            retrieved_context = []
            for passage_id, passage_rank, passage_score in zip(*results):
                retrieved_context.append(searcher.collection[passage_id])
                print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")

            msg = [{"role": "system", "content": persona_qa}] + messages
            msg.append({"role": "user", "content": json.dumps(retrieved_context)})
            qaresult = client.chat.completions.create(
                    model=llm_model,
                    messages=msg,
                    temperature=0,
                    seed=1
                )
            print("LLM 대답>>:")
            print(qaresult.choices[0].message.content)
    else:
        print("LLM 대답>>:")
        print(response_message.content)

In [ ]:
# 사용자와의 대화 메시지 목록을 정의합니다.
messages = [{"role": "user", "content": "광합성이 뭐니"},
            {"role": "user", "content": "메탄과 산소의 화학 반응"}
    # {"role": "user", "content": "안녕하세요, 저는 새로운 개 주인이고 방금 영국 마스티프를 입양했어요. 그를 돌보는 데 대해 몇 가지 팁을 줄 수 있나요?"},
    # # 사용자가 새로운 개 주인임을 밝히고, 개를 돌보는 데 대한 조언을 요청했습니다.
    # {"role": "assistant", "content": "물론이죠! 영국 마스티프 같은 대형견을 돌보는 것은 매우 보람찬 일입니다. 그들은 균형 잡힌 식단, 규칙적인 운동, 그리고 정기적인 수의사 검진이 필요합니다. 그들의 돌봄에 대해 구체적으로 궁금한 점이 있나요?"},
    # # 어시스턴트가 영국 마스티프 돌봄에 대한 일반적인 정보를 제공하며 추가적인 질문을 유도합니다.
    # {"role": "user", "content": "네, 특히 그의 식단에 대해 걱정이 되네요. 1살 영국 마스티프에게 얼마나 많은 양을 먹여야 하나요?"},
    # # 사용자가 구체적으로 영국 마스티프의 식단에 대한 질문을 하고 있습니다.
]

# 대화형 에이전트를 사용하여 반려 동물과 관련된 질의에 응답하도록 합니다.
conversation_search(messages, persona_function_calling, tools)


In [21]:
def answer_question(messages):
    # 함수 출력 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}  # 초기 응답 딕셔너리 생성 (질의, 상위 결과, 참조, 답변 필드 포함)

    # 질의 분석 및 검색 이외의 질의 대응을 위한 LLM 활용
    msg = [{"role": "system", "content": persona_function_calling}] + messages  # 시스템 메시지와 사용자 메시지를 결합하여 msg 생성
    try:
        result = client.chat.completions.create(  # OpenAI 클라이언트의 chat completion 생성 호출
            model=llm_model,  # 사용할 LLM 모델 설정
            messages=msg,  # 대화 메시지 전달
            tools=tools,  # 사용할 함수 리스트 전달
            # tool_choice={"type": "function", "function": {"name": "search"}},  # 사용 가능한 함수 중 하나를 선택하는 옵션 (주석 처리됨)
            temperature=0,  # 응답의 무작위성 설정 (0이면 더 결정적인 응답 생성)
            seed=1,  # 무작위성 제어를 위한 시드 설정
            timeout=10  # 요청 타임아웃 시간 설정 (초 단위)
        )
    except Exception as e:
        traceback.print_exc()  # 예외 발생 시 오류 추적 출력
        return response  # 오류 발생 시 초기화된 빈 응답 반환

    # # 검색이 필요한 경우 검색 호출 후 결과를 활용하여 답변 생성
    # if result.choices[0].message.tool_calls:  # LLM이 함수 호출을 요청한 경우
    #     tool_call = result.choices[0].message.tool_calls[0]  # 첫 번째 함수 호출 정보 가져오기
    #     function_args = json.loads(tool_call.function.arguments)  # 함수 인자를 JSON 형식으로 파싱
    #     standalone_query = function_args.get("standalone_query")  # standalone_query 추출

    # 생성된 응답 메시지를 변수에 저장합니다.
    response_message = result.choices[0].message

    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            # 도구 호출에 전달된 인자를 JSON 형식으로 파싱합니다.
            function_args = json.loads(tool_call.function.arguments)
            standalone_query = function_args.get("standalone_query")  # 독립적인 검색 쿼리를 추출합니다.


            # Baseline으로는 sparse_retrieve만 사용하여 검색 결과 추출
            results = searcher.search(standalone_query, k=3)  # sparse_retrieve 함수를 호출하여 검색 결과 가져오기

            response["standalone_query"] = standalone_query  # standalone_query 응답에 추가
            retrieved_context = []

            for passage_id, passage_rank, passage_score in zip(*results):  # 검색된 문서들을 순회
                retrieved_context.append(searcher.collection[passage_id])  # 검색된 문서의 content 필드 추가
                response["topk"].append(docid_list[passage_id])  # 검색된 문서의 docid 추가
                response["references"].append({"score": passage_score, "content": searcher.collection[passage_id]})  # 검색된 문서의 점수 및 내용 추가
                print(passage_id, passage_rank, passage_score)
                                    #print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")

                msg = [{"role": "system", "content": persona_qa}] + messages  # 질문 응답 프롬프트와 함께 메시지 결합
                msg.append({"role": "user", "content": json.dumps(retrieved_context)})  # assistant 역할로 검색된 컨텍스트 추가

        try:
            qaresult = client.chat.completions.create(  # OpenAI 클라이언트를 사용해 최종 답변 생성
                model=llm_model,  # 사용할 언어 모델 설정 (여기서는 gpt-3.5-turbo-1106)
                messages=msg,  # 대화에 사용할 메시지 리스트 전달
                temperature=0,  # 생성된 응답의 무작위성 설정 (0은 더 결정적인 응답 생성)
                seed=1,  # 무작위성 제어를 위한 시드 설정
                timeout=30  # 요청에 대한 타임아웃 설정 (30초)
            )
        except Exception as e:
            traceback.print_exc()  # 예외 발생 시 오류 추적 출력
            return response  # 오류 발생 시 초기화된 응답 반환

        response["answer"] = qaresult.choices[0].message.content  # 최종 생성된 답변 응답에 추가

    # 검색이 필요하지 않은 경우 바로 답변 생성
    else:
        response["answer"] = result.choices[0].message.content  # 검색 없이 생성된 답변 추가

    return response  # 최종 답변 반환


In [22]:
# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출 후 파일에 저장
def eval_rag(eval_filename, output_filename):  # 평가 파일과 출력 파일을 입력으로 받음
    with open(eval_filename) as f, open(output_filename, "w") as of:  # 평가 파일 읽기 모드, 출력 파일 쓰기 모드로 열기
        idx = 0  # 평가 데이터의 인덱스를 0으로 초기화
        for line in f:  # 평가 파일의 각 줄을 순회
            # if idx > 5:
            #     break  # 특정 개수 이상 평가를 제한할 때 사용 (현재 주석 처리됨)
            j = json.loads(line)  # 평가 데이터의 한 줄을 JSON 형식으로 로드
            print(f'Test {idx}\nQuestion: {j["msg"]}')  # 현재 인덱스와 질문 출력
            response = answer_question(j["msg"])  # 질문에 대한 답변 생성
            print(f'Answer: {response["answer"]}\n')  # 생성된 답변 출력

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}  # 평가 결과 저장할 딕셔너리 생성
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')  # 평가 결과를 JSON 형식으로 출력 파일에 저장
            idx += 1  # 인덱스를 1 증가

In [ ]:
# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
eval_rag("/data/ephemeral/home/upstage-ai-advanced-ir2/data/eval_copy.jsonl", "/data/ephemeral/home/upstage-ai-advanced-ir2/submission_test/ColBERT_submission.csv")  # 평가 파일 경로와 출력 파일명을 입력으로 하여 eval_rag 함수 호출